# Deep Q-Network (DQN)
---
In this notebook, you will implement a DQN agent with OpenAI Gym's LunarLander-v2 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [2]:
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)
print(env.reset())

State shape:  (8,)
Number of actions:  4
[-5.9156417e-04  1.4134574e+00 -5.9935719e-02  1.1277095e-01
  6.9228926e-04  1.3576316e-02  0.0000000e+00  0.0000000e+00]


Before running the next code cell, familiarize yourself with the code in **Step 2** and **Step 3** of this notebook, along with the code in `dqn_agent.py` and `model.py`.  Once you have an understanding of how the different files work together, 
- Define a neural network architecture in `model.py` that maps states to action values.  This file is mostly empty - it's up to you to define your own deep Q-network!
- Finish the `learn` method in the `Agent` class in `dqn_agent.py`.  The sampled batch of experience tuples is already provided for you; you need only use the local and target Q-networks to compute the loss, before taking a step towards minimizing the loss.

Once you have completed the code in `dqn_agent.py` and `model.py`, run the code cell below.  (_If you end up needing to make multiple changes and get unexpected behavior, please restart the kernel and run the cells from the beginning of the notebook!_)

You can find the solution files, along with saved model weights for a trained agent, in the `solution/` folder.  (_Note that there are many ways to solve this exercise, and the "solution" is just one way of approaching the problem, to yield a trained agent._)

In [3]:
from dqn_agent import Agent

agent = Agent(state_size=8, action_size=4, seed=0)

# watch an untrained agent
state = env.reset()
for j in range(200):
    action = agent.act(state)
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 
        
env.close()

### 3. Train the Agent with DQN

Run the code cell below to train the agent from scratch.  You are welcome to amend the supplied values of the parameters in the function, to try to see if you can get better performance!

In [ ]:
def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.997):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    solved = False
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}\tEpsilon: {:.2f}'.format(i_episode, np.mean(scores_window), eps), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            solved = True
            break
            
    if not solved:
        print('\nEnvironment NOT solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
        #torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
        
    return scores

#scores = dqn(n_episodes=1, max_t=10000)
scores = dqn(n_episodes=2000, eps_end=0.02)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 100	Average Score: -162.98	Epsilon: 0.74
Episode 200	Average Score: -139.09	Epsilon: 0.55
Episode 300	Average Score: -76.84	Epsilon: 0.414
Episode 400	Average Score: -25.96	Epsilon: 0.30
Episode 500	Average Score: 52.81	Epsilon: 0.227
Episode 600	Average Score: 115.24	Epsilon: 0.16
Episode 684	Average Score: 178.79	Epsilon: 0.13

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [5]:
# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

for i in range(5):
    state = env.reset()
    for j in range(2000):
        #action = agent.act(state)
        action_values = agent.get_act_values(state)
        print(action_values)
        action = np.argmax(action_values)
        env.render()
        state, reward, done, _ = env.step(action)
        if done:
            break 
            
env.close()

[[ -6.1343784  -4.302993  -11.254254   -6.8333254]]
[[ -6.04952    -4.235394  -10.713334   -6.7170606]]
[[ -5.959954   -4.1655498 -10.126934   -6.593653 ]]
[[-5.8923783 -4.1166983 -9.5522995 -6.489572 ]]
[[-5.8148203 -4.061303  -8.932613  -6.374224 ]]
[[-5.769402  -4.0356793 -8.229415  -6.2818723]]
[[-5.729984  -4.0178127 -7.495034  -6.192455 ]]
[[-5.699991 -4.015111 -6.723257 -6.115606]]
[[-5.81993   -4.216389  -6.3574653 -6.2703915]]
[[-6.0249257 -4.570038  -6.038325  -6.512243 ]]
[[-6.299939  -5.0031476 -5.797484  -6.778564 ]]
[[-6.6973066 -5.61906   -5.6612725 -7.220391 ]]
[[-7.085151  -6.2729125 -5.4568796 -7.6611032]]
[[-7.2179823 -6.4424644 -5.385754  -7.8013706]]
[[-7.280514  -6.5221953 -5.4270496 -7.850801 ]]
[[-7.230587  -6.421805  -5.577164  -7.7613015]]
[[-7.1735334 -6.3545337 -5.6623635 -7.66602  ]]
[[-7.1218715 -6.179598  -5.9450684 -7.5672235]]
[[-6.931469  -5.9054666 -6.134665  -7.3194923]]
[[-7.3104897 -6.593114  -5.9062405 -7.749416 ]]
[[-6.850889  -5.9809194 -6.17470

[[ -9.86072  -11.721837 -10.235626  -8.698738]]
[[ -9.966845 -12.003867  -9.848177  -8.987311]]
[[ -9.8425045 -11.9604     -9.245091   -9.024452 ]]
[[ -9.517742 -11.625702  -8.553363  -8.820181]]
[[ -9.594389 -11.164475 -10.846613  -8.162992]]
[[ -9.211222 -10.737072 -10.188982  -7.878346]]
[[ -8.842472 -10.340714  -9.506449  -7.625876]]
[[-8.453878 -9.912709 -8.816058 -7.339648]]
[[-8.12618  -9.569456 -8.194526 -7.131049]]
[[-7.793947  -9.226353  -7.5436077 -6.923762 ]]
[[-7.4587045 -8.879754  -6.886602  -6.7116704]]
[[-7.1626983 -8.588506  -6.30346   -6.554289 ]]
[[-7.1469984 -7.9713893 -8.482115  -5.798796 ]]
[[-6.8352118 -7.658422  -7.8260307 -5.628228 ]]
[[-6.561711  -7.39562   -7.2320194 -5.4995904]]
[[-6.2920213 -7.1445556 -6.6233797 -5.3822255]]
[[-5.978978  -6.8493943 -6.0679073 -5.2283998]]
[[-5.6619573 -6.543695  -5.588462  -5.061607 ]]
[[-5.3795533 -6.296921  -5.101403  -4.956227 ]]
[[-5.1553483 -6.0813828 -4.683206  -4.7570724]]
[[-5.2265553 -5.8573008 -5.495837  -4.568859

[[-5.987862 -6.227724 -5.763093 -6.634564]]
[[-5.8564253 -5.875935  -6.4855456 -6.554408 ]]
[[-6.0523667 -6.0913095 -5.9422836 -6.7112803]]
[[-6.045461  -5.881736  -6.1051035 -6.71206  ]]
[[-6.110996  -5.9961257 -5.3437963 -6.73526  ]]
[[-6.1368694 -5.8292675 -5.4287944 -6.7629995]]
[[-5.969005  -5.3612747 -6.091297  -6.6330085]]
[[-6.043002  -5.504676  -5.3604155 -6.6805987]]
[[-5.945301  -5.1515117 -5.8366623 -6.594875 ]]
[[-5.992345  -5.32232   -5.0451007 -6.6524944]]
[[-5.7836256 -4.7251234 -6.219445  -6.4528446]]
[[-5.8609276 -4.8975015 -5.5136414 -6.518161 ]]
[[-5.938967  -5.123297  -4.8119855 -6.628852 ]]
[[-5.685925  -4.339132  -6.3462954 -6.353659 ]]
[[-5.804638  -4.6015635 -5.7359505 -6.5026317]]
[[-5.8938136 -4.8525763 -5.0541267 -6.625829 ]]
[[-5.937753  -5.0556664 -4.324484  -6.6964936]]
[[-5.7158737 -4.4081836 -5.6003723 -6.438288 ]]
[[-5.798051 -4.659955 -4.927842 -6.552933]]
[[-5.9019213 -4.9712286 -4.25721   -6.68613  ]]
[[-5.6361547 -4.2494354 -5.664239  -6.4260297]]


[[-3.4911482 -2.125538  -3.8433    -4.659953 ]]
[[-3.2256703 -1.9287419 -2.6659596 -4.348933 ]]
[[-3.0078037 -1.7977794 -1.5381901 -4.0746098]]
[[-2.8091679 -1.2799612 -2.843037  -3.8274417]]
[[-2.579789  -1.1397736 -1.7772868 -3.58575  ]]
[[-2.4669726 -1.1776102 -0.7237436 -3.429943 ]]
[[-2.5242987 -1.0986413 -1.3467185 -3.4662871]]
[[-2.452791   -1.212342   -0.16280527 -3.3802857 ]]
[[-2.614483  -1.4206628 -0.3370878 -3.5136876]]
[[-2.81431   -1.6399378 -0.6942131 -3.6695027]]
[[-2.950392  -1.7343228 -0.9810445 -3.7225733]]
[[-3.046664  -1.8424146 -0.7790615 -3.7786198]]
[[-3.0905595  -1.9055922  -0.37400454 -3.790935  ]]
[[-3.1345057  -1.9660131  -0.47412223 -3.796142  ]]
[[-3.9555712 -2.8407886 -2.3548834 -4.515676 ]]
[[-5.0298376 -3.988435  -4.4193783 -5.6143646]]
[[-6.5876875 -6.050549  -6.091474  -7.285119 ]]
[[-8.031569 -8.014593 -7.352091 -8.866844]]
[[-8.457252  -8.602943  -7.6776633 -9.313976 ]]
[[-8.411232  -8.47223   -7.8099155 -9.231806 ]]
[[-8.474466  -8.541301  -7.87618

[[-7.73719   -7.8272862 -7.0525274 -8.669974 ]]
[[-7.525218 -7.156984 -8.065455 -8.42545 ]]
[[-7.663274  -7.423617  -7.3861127 -8.612612 ]]
[[-7.404707 -6.67926  -8.562852 -8.300038]]
[[-7.587437  -6.999536  -7.9648943 -8.53926  ]]
[[-7.7131324 -7.2668076 -7.2726974 -8.715829 ]]
[[-7.8619847 -7.564912  -6.618715  -8.917857 ]]
[[-7.8531985 -7.3641224 -7.123564  -8.905711 ]]
[[-7.5073876 -6.4161286 -8.63462   -8.494592 ]]
[[-7.4835887 -6.5134354 -7.9221334 -8.492383 ]]
[[-7.4495683 -6.6131396 -7.147607  -8.477717 ]]
[[-7.4779515 -6.791072  -6.5402193 -8.538995 ]]
[[-7.322434  -6.3137164 -7.6284347 -8.386244 ]]
[[-7.3948226 -6.5219016 -7.1637263 -8.502419 ]]
[[-7.45486   -6.7583055 -6.5996456 -8.599833 ]]
[[-7.02882   -5.8494506 -8.196444  -8.168813 ]]
[[-7.061735  -6.0125217 -7.6405516 -8.236453 ]]
[[-7.0951324 -6.189716  -7.1383033 -8.303541 ]]
[[-7.048211  -6.2642994 -6.554993  -8.280652 ]]
[[-6.9944515 -6.334913  -5.94471   -8.247828 ]]
[[-6.8808084 -6.022484  -6.779655  -8.148652 ]]


...

[[-6.062154  -6.601744  -4.415053  -4.9733806]]
[[-6.1056356 -6.397241  -5.4840307 -4.546002 ]]
[[-5.8810043 -6.189853  -4.668282  -4.552731 ]]
[[-5.650403  -5.9761896 -3.9755867 -4.5564795]]
[[-5.579893  -5.691657  -4.668085  -4.0731983]]
[[-5.2996764 -5.4632926 -3.6673005 -4.055736 ]]
[[-5.395055 -5.524248 -3.961023 -3.990203]]
[[-5.3750143 -5.5143924 -3.8256538 -3.9464722]]
[[-5.312285  -5.4528317 -3.6339223 -3.8486295]]
[[-5.232229  -5.3400707 -3.5480855 -3.65587  ]]
[[-5.408065  -5.438675  -4.246759  -3.5989141]]
[[-5.1309524 -5.218686  -3.2317212 -3.5723968]]
[[-5.147494  -5.1742296 -3.4998791 -3.4340758]]
[[-4.9135237 -5.016628  -2.5214102 -3.4724112]]
[[-5.0918393 -5.191033  -2.827947  -3.5809073]]
[[-5.1487412 -5.23066   -2.962301  -3.6102371]]
[[-5.3724165 -5.4708066 -3.3688266 -3.9167347]]
[[-5.477859  -5.470833  -4.2594376 -3.8874955]]
[[-6.119366  -6.412298  -4.5698066 -4.9826922]]
[[-6.408225  -6.743262  -5.3598614 -5.3310547]]
[[-7.076639 -7.711721 -5.759083 -6.481836]]


[[-7.1559596 -9.075312  -5.4446225 -8.088924 ]]
[[-7.388668 -9.292127 -5.344452 -8.372297]]
[[-7.2517133 -8.935872  -5.2563124 -8.193037 ]]
[[-7.2038827 -8.746324  -5.4240947 -8.1338   ]]
[[-7.357245 -8.834668 -5.404231 -8.294162]]
[[-7.618304 -9.048092 -5.197653 -8.575297]]
[[-7.4454756 -8.627262  -5.3098283 -8.356948 ]]
[[-7.44551   -8.51433   -5.3029027 -8.344306 ]]
[[-7.35388  -8.233591 -5.387755 -8.24107 ]]
[[-7.27141   -7.834819  -5.8560534 -8.164919 ]]
[[-7.0443654 -7.0617404 -6.7661986 -7.960831 ]]
[[-6.9389176 -6.5465083 -7.4218187 -7.8702846]]
[[-6.9529233 -6.565765  -6.521965  -7.8374085]]
[[-6.819149  -5.857513  -8.11887   -7.7488303]]
[[-6.9420376 -5.992257  -7.5392327 -7.8493495]]
[[-7.13027   -6.2905884 -6.9496655 -8.013935 ]]
[[-7.3392196 -6.6624603 -6.3767886 -8.22358  ]]
[[-7.392155 -6.237955 -8.314902 -8.344664]]
[[-7.6003942 -6.613629  -7.7352247 -8.549535 ]]
[[-7.862479  -7.1138225 -7.274346  -8.873072 ]]
[[-7.914331  -7.325246  -6.7045584 -8.945421 ]]
[[-7.626005 

[[-6.564984  -5.7856617 -8.198336  -7.5198903]]
[[-6.449752  -5.764437  -7.466856  -7.4075336]]
[[-6.160772  -5.490924  -6.5547457 -7.094024 ]]
[[-5.8650107 -5.2024546 -5.6675215 -6.773338 ]]
[[-5.5678368 -5.1517625 -1.2704284 -6.3602824]]
[[-5.4799795 -4.111223  -6.9177356 -6.333703 ]]
[[-4.9701505 -3.661447  -7.8445225 -5.948742 ]]
[[-4.9355354 -3.8446841 -6.9409094 -5.934622 ]]
[[-4.8954215 -4.028294  -6.328888  -5.9276547]]
[[-4.8694286 -4.26033   -5.641983  -5.9365516]]
[[-3.186325  -1.6657151 -2.8728473 -3.6939774]]
[[-2.9320006 -1.4792497 -1.8104568 -3.34791  ]]
[[-2.7577667 -1.3845958 -0.6580307 -3.0888429]]
[[-4.6601415 -4.786862  -4.16517   -5.8033676]]
[[-2.2975857 -0.7292915 -1.2767673 -2.6150508]]
[[-2.2517514  -0.81200886 -0.3887071  -2.498097  ]]
[[-2.0395722  -0.42123544 -1.6502275  -2.3294454 ]]
[[-2.0003505  -0.5028411  -0.70380205 -2.199741  ]]
[[-1.9679228  -0.66361266  0.40911514 -2.1280212 ]]
[[-1.8750677  -0.53939456  0.21971707 -1.9926139 ]]
[[-1.9856042  -0.732

[[-3.988106  -2.7003708 -2.57948   -4.6907005]]
[[-3.7187178 -2.2733953 -2.8377745 -4.355295 ]]
[[-3.8222923 -2.608769  -2.1874058 -4.5510287]]
[[-3.7149627 -2.4247727 -2.532583  -4.3737006]]
[[-3.8502474 -2.7903962 -1.9557478 -4.597259 ]]
[[-3.9300473 -2.9385672 -1.9821079 -4.6705685]]
[[-3.9858272 -3.0000386 -2.0857642 -4.6830163]]
[[-4.113499  -3.206662  -1.9150875 -4.8171077]]
[[-4.1492043 -3.2404966 -2.280887  -4.8075714]]
[[-4.037243  -2.961584  -2.9106395 -4.5740676]]
[[-4.336972  -3.3681374 -3.1979744 -4.849051 ]]
[[-4.637409  -3.5724688 -4.4572387 -4.9738383]]
[[-5.976711  -5.397254  -5.7088213 -6.4614077]]
[[-7.3370204 -7.306379  -6.96633   -7.984174 ]]
[[-8.117741 -8.331106 -7.923152 -8.815047]]
[[-7.8087363 -7.8770943 -8.318286  -8.378735 ]]
[[-8.376675  -8.8829    -7.8458586 -8.997611 ]]
[[-8.450208 -9.143249 -7.638234 -9.032637]]
[[-8.686985  -9.509323  -7.7096252 -9.253097 ]]
[[-8.459178  -9.267162  -7.9452105 -8.938763 ]]
[[-8.075546 -8.726463 -8.209695 -8.453683]]
[[-8

[[-4.539472  -3.847251  -3.4066045 -5.149157 ]]
[[-4.52117   -3.7832937 -3.8670466 -5.1671343]]
[[-4.320784  -3.6550155 -3.0647094 -4.932846 ]]
[[-4.125486  -3.197082  -4.293263  -4.7957473]]
[[-3.9549189 -3.1089306 -3.5144436 -4.5943384]]
[[-3.6801703 -2.89195   -2.4014838 -4.2608395]]
[[-3.6165802 -2.5431087 -3.9420044 -4.22433  ]]
[[-3.3314297 -2.3238907 -2.75581   -3.8725781]]
[[-3.196633  -2.3231184 -1.6898558 -3.6821938]]
[[-3.266852  -2.3582835 -2.2521603 -3.7438726]]
[[-3.1638525 -2.128551  -2.8070724 -3.6504388]]
[[-3.114946  -2.2540662 -1.9023254 -3.575945 ]]
[[-3.0383878 -2.0096967 -2.5676239 -3.512951 ]]
[[-3.1926851 -2.451647  -1.6223819 -3.6674871]]
[[-3.096745 -2.226638 -2.09948  -3.574173]]
[[-2.908855  -1.8549598 -2.6447113 -3.3908405]]
[[-3.0599432 -2.283859  -1.6828725 -3.5415578]]
[[-2.9231565 -1.9800664 -2.3085592 -3.4107013]]
[[-3.2439604 -2.6311793 -1.7757604 -3.7153916]]
[[-4.034111  -3.5573556 -3.0663524 -4.396423 ]]
[[-5.020085  -4.596217  -5.2972794 -5.374855

### 5. Explore

In this exercise, you have implemented a DQN agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task with discrete actions!
- You may like to implement some improvements such as prioritized experience replay, Double DQN, or Dueling DQN! 
- Write a blog post explaining the intuition behind the DQN algorithm and demonstrating how to use it to solve an RL environment of your choosing.  